In [ ]:
import torch
print(torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"

False


In [ ]:
# !gdown "1GStA7e0LgjjqYmoXFH6I8tsgjYy3x-LT"  # OSACT2022-sharedTask-test-tweets.csv
# !wget "https://alt.qcri.org/resources1/OSACT2022/OSACT2022-sharedTask-test-taskB-gold-labels.txt"  # OSACT2022-sharedTask-test-taskB-gold-labels.txt
!wget "https://alt.qcri.org/resources/OSACT2022/OSACT2022-sharedTask-test-tweets.txt"
#labels for test
!wget "https://alt.qcri.org/resources1/OSACT2022/OSACT2022-sharedTask-test-taskB-gold-labels.txt"
!gdown "165kzfZDsRTZAAfZKedeZiUlKzMcHNgPd"  # arabic stop words
!gdown "1WdgbvqDYIa-g5ijjsz5zb-3lVvUXUtmS&confirm=t"  # qarib pretrained model
!gdown "1foNTGFjhWAxS-_SfF7rga80UmFT7BDJ0&confirm=t"  # fasttext-ar-vectors-150.bin

--2024-07-27 15:00:16--  https://alt.qcri.org/resources/OSACT2022/OSACT2022-sharedTask-test-tweets.txt
Resolving alt.qcri.org (alt.qcri.org)... 37.186.61.205
Connecting to alt.qcri.org (alt.qcri.org)|37.186.61.205|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://alt.qcri.org/resources1/OSACT2022/OSACT2022-sharedTask-test-tweets.txt [following]
--2024-07-27 15:00:17--  http://alt.qcri.org/resources1/OSACT2022/OSACT2022-sharedTask-test-tweets.txt
Connecting to alt.qcri.org (alt.qcri.org)|37.186.61.205|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://alt.qcri.org/resources1/OSACT2022/OSACT2022-sharedTask-test-tweets.txt [following]
--2024-07-27 15:00:17--  https://alt.qcri.org/resources1/OSACT2022/OSACT2022-sharedTask-test-tweets.txt
Connecting to alt.qcri.org (alt.qcri.org)|37.186.61.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 348638 (340K) [text/plain]
Saving to: ‘OSAC

In [ ]:
columns = [
    "id",
    "tweet",
    "off_label",  # Offensive & NotOffensive
    "hs_label",   # Hate Speech [Gender, Race, Ideology, Social Class, Religion, Disability]
    "vlg_label",  # Vulgar
    "violence_label" # Violence
    ]

In [ ]:
import pandas as pd
import numpy as np

test = pd.read_csv("OSACT2022-sharedTask-test-tweets.csv")
test.columns = ["id", "tweet"]
label_hate = pd.read_csv("OSACT2022-sharedTask-test-taskB-gold-labels.txt")
label_hate.columns = ["HS_label"]

test = test.join(label_hate)
test.drop("id", axis=1, inplace=True)
print(test.info())

test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet     2540 non-null   object
 1   HS_label  2540 non-null   object
dtypes: object(2)
memory usage: 39.8+ KB
None


,tweet,HS_label
0,مشفتش العرض بتاعهم لا مش مهتمة لا😩🐸😂 URL,NOT_HS
1,RT @USER عندما تكون لوحدك تحس انك لحالك صح 😊<L...,NOT_HS
2,RT @USER ماشاء الله الرجال باين عليه محترم <LF...,NOT_HS
3,@USER شسالفة احد يفهمني 😤,NOT_HS
4,@USER اقووووول استريح عاد احتفالاتنا تحط اغاني...,NOT_HS
...,...,...
2535,قله حيا وين اهلهم ذولي الله لايبلانا لهالدرجه ...,NOT_HS
2536,RT @USER ثم الطحلبه🐸🐸🐸 URL,NOT_HS
2537,يا وجه الله 😷 من اليوم ورايح شاورما انسى 🔪 URL,NOT_HS
2538,@USER متخلف حتى الحلال حرمتوه 😷,NOT_HS


In [ ]:
label2id = {"NOT_HS": 0, "HS": 1}
id2label = {0: "NOT_HS", 1: "HS"}

In [ ]:
test.rename(columns={"HS_label":"class"}, inplace=True)
test.head(1)

,tweet,class
0,مشفتش العرض بتاعهم لا مش مهتمة لا😩🐸😂 URL,NOT_HS


In [ ]:
arabic_stop_words=[]
with open ('./Arabic_stop_words.txt',encoding='utf-8') as f :
    for word in f.readlines() :
        arabic_stop_words.append(word.split("\n")[0])

In [ ]:
!pip install pyarabic
!pip install farasapy
!pip install transformers[torch]
!pip install Keras-Preprocessing

! git clone https://github.com/facebookresearch/fastText.git
! cd fastText && sudo pip install .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import fasttext
import fasttext.util
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-ar-vectors", filename="model.bin")
# model_path = "./fasttext-ar-vectors-150.bin"
model_fasttext = fasttext.load_model(model_path)
# model_fasttext = fasttext.util.reduce_model(model_fasttext, 150) # reduce embeddings dimension to 150 from 300; requires a huge memory notebook
# model_fasttext.save_model("/content/drive/MyDrive/Colab Notebooks/text-aml/hate-speech-ds/fasttext-ar-vectors-150.bin")
print(len(model_fasttext.words))
model_fasttext['bread'].shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/7.25G [00:00<?, ?B/s]

2000000


(300,)

In [ ]:
import functools, operator
import logging

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.isri import ISRIStemmer
import string
import re
from bs4 import BeautifulSoup
nltk.download('stopwords')


tok = WordPunctTokenizer()

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    return re.sub(arabic_diacritics, '', text)


def remove_punctuations(text):
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
    english_punctuations = string.punctuation
    punctuations_list = arabic_punctuations + english_punctuations
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    # return re.sub(r'(.)\1+', r'\1', text)     # keep only 1 repeat
    return re.sub(r'(.)\1+', r'\1\1', text)  # keep 2 repeat

def remove_stop_words(text):
    #nltk.download('stopwords')
    englishStopWords = stopwords.words('english')

    all_stopwords = set(englishStopWords + arabic_stop_words)

    word_list = nltk.tokenize.wordpunct_tokenize(text.lower())
    word_list = [ w for w in word_list if not w in all_stopwords ]
    return (" ".join(word_list)).strip()

def get_root(text):
    word_list = nltk.tokenize.wordpunct_tokenize(text.lower())
    result = []
    arstemmer = ISRIStemmer()
    for word in word_list: result.append(arstemmer.stem(word))
    return (' '.join(result)).strip()

def clean_tweet(text):
    text = re.sub(r'([@A-Za-z0-9_]+)|#|http\S+', ' ', text) # removes non arabic letters
    text = re.sub(r'ـــــــــــــ', '', text) # removes non arabic letters
    return text




def clean_str(text):
    text = clean_tweet(text)
    # text = normalize_arabic(text)
    text = remove_punctuations(text) ###
    text = remove_diacritics(text)
    text = remove_repeating_char(text) ###
    # text = remove_stop_words(text) ###


    text = text.replace('وو', 'و') ###
    text = text.replace('يي', 'ي') ###
    text = text.replace('اا', 'ا') ###

    # text = get_root(text) ###

    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    pat1 = r'@[A-Za-z0-9]+'
    pat2 = r'https?://[A-Za-z0-9./]+'
    combined_pat = r'|'.join((pat1, pat2))
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped

    words = tok.tokenize(clean)
    return (" ".join(words)).strip()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
PREFIX_LIST = [
    "ال",
    "و",
    "ف",
    "ب",
    "ك",
    "ل",
    "لل",
    "\u0627\u0644",
    "\u0648",
    "\u0641",
    "\u0628",
    "\u0643",
    "\u0644",
    "\u0644\u0644",
    "س",
]
SUFFIX_LIST = [
    "ه",
    "ها",
    "ك",
    "ي",
    "هما",
    "كما",
    "نا",
    "كم",
    "هم",
    "هن",
    "كن",
    "ا",
    "ان",
    "ين",
    "ون",
    "وا",
    "ات",
    "ت",
    "ن",
    "ة",
    "\u0647",
    "\u0647\u0627",
    "\u0643",
    "\u064a",
    "\u0647\u0645\u0627",
    "\u0643\u0645\u0627",
    "\u0646\u0627",
    "\u0643\u0645",
    "\u0647\u0645",
    "\u0647\u0646",
    "\u0643\u0646",
    "\u0627",
    "\u0627\u0646",
    "\u064a\u0646",
    "\u0648\u0646",
    "\u0648\u0627",
    "\u0627\u062a",
    "\u062a",
    "\u0646",
    "\u0629",
]


# the never_split list is used with the transformers library
_PREFIX_SYMBOLS = [x + "+" for x in PREFIX_LIST]
_SUFFIX_SYMBOLS = ["+" + x for x in SUFFIX_LIST]
NEVER_SPLIT_TOKENS = list(set(_PREFIX_SYMBOLS + _SUFFIX_SYMBOLS))

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


In [ ]:
import torch
from transformers import BertForSequenceClassification, AutoConfig, AutoTokenizer

# Define model name and configuration
model_name = "qarib/bert-base-qarib"
num_labels = 2

# Load model configuration
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels, output_attentions=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, do_basic_tokenize=True, model_max_length=64)
tokenizer.max_len = 64

# Initialize the model with the configuration
model = BertForSequenceClassification.from_pretrained(model_name, config=config)

# Load the saved state dictionary
device = torch.device('cpu')
model.load_state_dict(torch.load('/content/drive/MyDrive/model.pth', map_location=device))

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at qarib/bert-base-qarib and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully.


In [ ]:
# Example inference
inputs = tokenizer("انت غريب و سئ", return_tensors="pt", padding=True, truncation=True, max_length=64)
inputs = {key: val.to(device) for key, val in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

print("Predictions:", predictions)


Predictions: tensor([0])


In [ ]:
def classsify_tweets(tweet):
    df = pd.DataFrame({"tweet": tweet})
    df['clean_tweet'] = df['tweet'].apply(lambda x: clean_str(x))

    dev_df = pd.DataFrame({
        'id':range(len(df)),
        'text': df["clean_tweet"]
        })

    test_example = SingleSentenceClassificationProcessor(mode='classification')
    test_example.add_examples(texts_or_text_and_labels=dev_df['text'], overwrite_examples = True)

    test_features = test_example.get_features(tokenizer = tokenizer, max_length =64)

    input_ids = [i.input_ids for i in test_features]
    attention_masks = [i.attention_mask for i in test_features]

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    # Put the model in an evaluation state
    model.eval()

    # Transfer model to GPU
    model.to(device)

    torch.cuda.empty_cache() # empty the gpu memory
    # Transfer the batch to gpu
    inputs = inputs.to(device)
    masks = masks.to(device)

    # Run inference on the example
    output = model(inputs, attention_mask=masks)["logits"]
    # Transfer the output to CPU again and convert to numpy
    output = output.cpu().detach().numpy()

    return output

In [ ]:
from transformers import pipeline
unmasker_MARBERT = pipeline('fill-mask', model='UBC-NLP/MARBERT', top_k=50)

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def light_preprocess(text):
    text = clean_tweet(text)
    # text = normalize_arabic(text)
    text = remove_punctuations(text) ###
    text = remove_diacritics(text)
    text = remove_repeating_char(text) ###
    text = text.replace('وو', 'و') ###
    text = text.replace('يي', 'ي') ###
    text = text.replace('اا', 'ا') ###
    return text

nltk.download('stopwords')
englishStopWords = stopwords.words('english')
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

all_stopwords = set(englishStopWords + arabic_stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
size = len(test)
print("size of test set:", size)
correct_class_tweets = []
correct_class = []
for i in range(0, size):
    txt = test['tweet'].astype('U')[i]
    cls = test['class'][i]
    label = id2label[np.argmax(classsify_tweets([txt]), axis=1)[0]]
    if label == cls and label == "HS":
        correct_class_tweets.append(txt)
        correct_class.append(cls)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


size of test set: 2540


In [ ]:
print("len(correctly classified hatespeach tweets):", len(correct_class_tweets))
print("Accuracy of the hatespeach class:", len(correct_class_tweets)/len(test[test["class"]=='HS']))

len(correctly classified hatespeach tweets): 140
Accuracy of the hatespeach class: 0.5166051660516605


In [ ]:


import pandas as pd

# Create a DataFrame from the lists
df = pd.DataFrame({'tweet': correct_class_tweets, 'class': correct_class})


# Save the DataFrame to a CSV file
df.to_csv('correctly_classified_hate_tweets.csv', index=False)


In [ ]:
from scipy.spatial import distance
from farasa.stemmer import FarasaStemmer
frasa_stemmer = FarasaStemmer(interactive=True)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [02:52<00:00, 1.40MiB/s]


[2024-07-27 15:56:47,841 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 20.1 MB/s eta 0:00:00


In [ ]:
import emoji
def select_best_replacement(pos, x_cur, verbose=False):
    """ Select the most effective replacement to word at pos (pos)
    in (x_cur)"""

    if bool(emoji.emoji_count(x_cur.split()[pos])):
        return None

    embedding_masked_word = model_fasttext[x_cur.split()[pos]]

    x_masked = (" ".join(x_cur.split()[:pos]) + " [MASK] " + " ".join(x_cur.split()[pos + 1:])).strip()
    unmasked_seq = unmasker_MARBERT(x_masked)[:20]

    max_sim = -1
    best_perturb_dict = {}
    for seq in unmasked_seq:
        if frasa_stemmer.stem(seq['token_str']) in frasa_stemmer.stem(x_cur.split()[pos]):
            # print(f"Skip this candidate word: ({seq['token_str']}). It has the same root as the masked word.")
            continue
        if seq['token_str'] in punctuations_list or pos >= len(seq["sequence"].split()):
            continue
        embedding_masked_word_new = model_fasttext[seq['token_str']]
        if np.sum(embedding_masked_word) == 0 or np.sum(embedding_masked_word_new) == 0:
            continue
        if verbose: print("New word: ", seq['token_str'])
        sim = 1 - distance.cosine(embedding_masked_word, embedding_masked_word_new)
        if sim > max_sim:
            max_sim = sim
            best_perturb_dict["sim"] = sim
            best_perturb_dict["Masked word"] = x_cur.split()[pos]
            best_perturb_dict["New word"] = seq['token_str']
            best_perturb_dict["New seq"] =  x_cur.replace(x_cur.split()[pos], seq['token_str'])  # seq["sequence"]

    return best_perturb_dict.get("New seq", None)

In [ ]:
# Define the number of tweets to process
num_tweets_to_process = min(20, len(correct_class_tweets))

# Initialize perturbation counter
perturb_counter = 0

for tweet_ix in range(num_tweets_to_process):
    tweet = correct_class_tweets[tweet_ix]
    print("Tweet index: ", tweet_ix)

    x_adv = light_preprocess(tweet)
    x_len = len(x_adv.split())
    orig_class = np.argmax(classsify_tweets([x_adv]), axis=1)[0]
    orig_label = id2label[orig_class]
    print(f"Original tweet: {x_adv} : Original label: {orig_label}.")
    splits = len(x_adv.split())
    perturbed_flag = False
    for split_ix in range(splits):
        perturbed = select_best_replacement(split_ix, x_adv)
        if perturbed:
            new_class = np.argmax(classsify_tweets([perturbed]), axis=1)[0]
            if orig_class != new_class:
                print(f"Perturbed tweet: {perturbed} : New label: {id2label[new_class]}.")
                print(10 * "==")
                if not perturbed_flag:
                    perturb_counter += 1
                perturbed_flag = True
    if not perturbed_flag:
        print(10 * "==")

print(f"Successful perturbation {perturb_counter} out of {num_tweets_to_process}.")


Tweet index:  0
Original tweet: وش ذا والله لو مو شايفات رجال حلوين بحياتهم استغفر الله قله حياء وتخلف 👎🏻👎🏻👎🏻😡😡  بايع الكليجا : Original label: HS.


[2024-07-27 16:10:06,621 - farasapy_logger - ERROR]: pipe broke! error code and message: [[Errno 32] Broken pipe]. reinitailize the process.., This may take sometime depending on the running task


Perturbed tweet: وش ذا والله لو مو شايفات الناس حلوين بحياتهم استغفر الله قله حياء وتخلف 👎🏻👎🏻👎🏻😡😡  بايع الكليجا : New label: NOT_HS.
Perturbed tweet: وش ذا والله لو مو شايفات رجال حلوين بحياتهم استغفر الله قله حياء وقرف 👎🏻👎🏻👎🏻😡😡  بايع الكليجا : New label: NOT_HS.
Tweet index:  1
Original tweet: والله عيب اللي صار بغض النظر جميل او شين وين كرامتك وحيائك  يعني بعض البنات اللي حاولو يعملو لفت انتباه وش تتوقعين منه اتجاهك 😷 انا ومالي دخل تقرفت من بعض الهمج اللي بالفيديو  بايع الكليجا  الجنادرية : Original label: HS.
Perturbed tweet: والله عيب اللي صار بغض النظر جميل او شين وين كرامتك وحيائك  يعني اغلب البنات اللي حاولو يعملو لفت انتباه وش تتوقعين منه اتجاهك 😷 انا ومالي دخل تقرفت من اغلب الهمج اللي بالفيديو  بايع الكليجا  الجنادرية : New label: NOT_HS.
Perturbed tweet: والله عيب اللي صار بغض النظر جميل او شين وين كرامتك وحيائك  يعني بعض الشباب اللي حاولو يعملو لفت انتباه وش تتوقعين منه اتجاهك 😷 انا ومالي دخل تقرفت من بعض الهمج اللي بالفيديو  بايع الكليجا  الجنادرية : New label: NOT_HS.
Pert